In [4]:
import pandas as pd
import geopandas as gpd

from bs4 import BeautifulSoup


In [5]:
def extract(lst):
    return [item[-1] for item in lst]


In [6]:
file_path='raw/impots_suite.html'

soup = BeautifulSoup(open(file_path), "html.parser")


In [7]:
impots = soup.find_all("ol")
i = impots[0]
rows =[]
for impot in impots:
    
    row = {'name':impot.find(class_='name').get_text(),
           'address':impot.find(class_="address").get_text(),
           "tel":impot.find(class_="phone").get_text()
          }
    rows.append(row)

In [8]:
df = pd.DataFrame.from_dict(rows)

In [9]:
df

,name,address,tel
0,. IMPOTS Direction,"\n 3,REGGANE,Cité El Kaous Labyad,Adrar",049 32 02 02
1,. IMPOTS Direction,"\n 4,REGGANE,Cité El Kaous Labyad,Adrar",049 32 02 07
2,. IMPOTS Direction,"\n 2,REGGANE,Cité El Kaous Labyad,Adrar",049 32 03 20
3,. IMPOTS Direction,"\n AOULEF,Cité Centre Ville,Adrar",049 32 71 96
4,. IMPOTS Direction,"\n AOULEF,Cité Centre Ville,Adrar",049 32 78 27
...,...,...,...
320,C.P.I MOHAMMADIA CENTRE DE PROXIMITE DES IMPOTS,"\n MOHAMMADIA MASCARA,Rue Emir AEK Prol...",045 94 14 33
321,INSPECTION TIGHENNIF IMPOTS Direction,"\n TIGHENNIF,Boulevard Des Martyrs,Masc...",045 97 23 79
322,RECETTE TIGHENNIF IMPOTS Direction,"\n TIGHENNIF,Boulevard Des Martyrs,Masc...",045 97 23 83
323,Recette des impots EL BORDJ IMPOTS Direction,"\n TIGHENNIF,Rue 1er Novembre 1954,Masc...",045 97 24 06


In [10]:
impots =df.groupby(['address'])['tel'].apply(' - '.join).reset_index()


In [11]:
impots

,address,tel
0,"\n 1,SAIDA,Rue Allel Medeghri,Saida",048 37 31 27
1,"\n 10,MECHERIA,Cité 12 Logements Admini...",049 51 42 12
2,"\n 2,REGGANE,Cité El Kaous Labyad,Adrar",049 32 03 20
3,"\n 27,MECHERIA,Cité Badr,Naama",049 51 41 26
4,"\n 3,BECHAR,Rue Zaoui Dieb,Bechar",049 22 11 62
...,...,...
126,"\n TIMIMOUN,Cité Hassi Ghambou,Adrar",049 90 45 51 - 049 90 45 59
127,"\n TISSEMSILT,60 Bureaux Wilaya,Tissems...",046 50 33 48 - 046 57 01 23 - 046 57 01 35 ...
128,"\n TISSEMSILT,Cité 300 Logements,Tissem...",046 56 91 70 - 046 56 91 75 - 046 56 91 78
129,"\n TISSEMSILT,Cité 320 Logements,Tissem...",046 57 08 61 - 046 57 08 63 - 046 57 08 66


In [12]:
merge = pd.merge(impots,df,on="address")
merge = merge.drop_duplicates(subset='address')

In [13]:
merge.drop(columns=['tel_y'],inplace=True)

In [14]:
merge['wilaya'] =  extract(merge['address'].str.split(','))

merge['address'] = merge['address'].str.replace('xyz,','',case=False).str.strip()

merge.rename(columns={'tel_x':'tel'},inplace=True)

In [15]:
merge

,address,tel,name,wilaya
0,"1,SAIDA,Rue Allel Medeghri,Saida",048 37 31 27,. IMPOTS DE SAIDA,Saida
1,"10,MECHERIA,Cité 12 Logements Administrati,Naama",049 51 42 12,. IMPOTS DIRECTION,Naama
2,"2,REGGANE,Cité El Kaous Labyad,Adrar",049 32 03 20,. IMPOTS Direction,Adrar
3,"27,MECHERIA,Cité Badr,Naama",049 51 41 26,. IMPOTS DIRECTION,Naama
4,"3,BECHAR,Rue Zaoui Dieb,Bechar",049 22 11 62,. IMPOTS,Bechar
...,...,...,...,...
310,"TIMIMOUN,Cité Hassi Ghambou,Adrar",049 90 45 51 - 049 90 45 59,. IMPOTS Direction,Adrar
312,"TISSEMSILT,60 Bureaux Wilaya,Tissemsilt",046 50 33 48 - 046 57 01 23 - 046 57 01 35 ...,. IMPOTS Direction,Tissemsilt
318,"TISSEMSILT,Cité 300 Logements,Tissemsilt",046 56 91 70 - 046 56 91 75 - 046 56 91 78,. IMPOTS Direction,Tissemsilt
321,"TISSEMSILT,Cité 320 Logements,Tissemsilt",046 57 08 61 - 046 57 08 63 - 046 57 08 66,. IMPOTS Direction,Tissemsilt


In [16]:
merge.to_csv('impots_suite.csv')

In [78]:
merge.wilaya

106

In [68]:
path = '../../dz-admin/wilayas_48.csv'
wilayas = pd.read_csv(path)

In [73]:
import fuzzy_pandas as fpd
wilayas

,num,code,nom,nom_maj,nom_ar
0,0,1,Adrar,ADRAR,أدرار
1,1,2,Chlef,CHLEF,الشلف
2,2,3,Laghouat,LAGHOUAT,اﻷغواط
3,3,4,Oum El Bouaghi,OUM EL BOUAGHI,أم البواقي
4,4,5,Batna,BATNA,باتنة
5,5,6,Béjaïa,BEJAIA,بجاية
6,6,7,Biskra,BISKRA,بسكرة
7,7,8,Béchar,BECHAR,بشار
8,8,9,Blida,BLIDA,البليدة
9,9,10,Bouira,BOUIRA,البويرة


In [83]:
merge.wilaya.drop_duplicates()


0                    Blida 
7                   Bouira 
8           Oum El Bouaghi 
10                  Bejaia 
15                  Biskra 
16                Ghardaia 
23             Constantine 
25              Souk Ahras 
26                  Tipaza 
27              Mostaganem 
28                    Oran 
30                   Alger 
39                Laghouat 
41                  Skikda 
62                 Ouargla 
71               Boumerdes 
72               Khenchela 
81                  Guelma 
85                 Tebessa 
96                   Batna 
100                  Setif 
113                  Jijel 
114                   Mila 
145         Ain Temouchent 
167                 Illizi 
193                 Annaba 
196                  Medea 
199              Ain Defla 
219             Tizi Ouzou 
222                  MSila 
237                 Djelfa 
242            Tamanrasset 
363                Tlemcen 
420                El Tarf 
518                El Oued 
532     Bordj Bou Ar

In [81]:
impots_wilaya = pd.DataFrame(data=merge.wilaya.drop_duplicates(),columns=['wilaya'])

result = fpd.fuzzy_merge(wilayas, impots_wilaya,
                left_on=['nom'],
                right_on=['wilaya'],
                ignore_case=True,
                method='levenshtein',        
                keep='all')
result

,num,code,nom,nom_maj,nom_ar,wilaya
0,1,2,Chlef,CHLEF,الشلف,Chlef
1,2,3,Laghouat,LAGHOUAT,اﻷغواط,Laghouat
2,3,4,Oum El Bouaghi,OUM EL BOUAGHI,أم البواقي,Oum El Bouaghi
3,4,5,Batna,BATNA,باتنة,Batna
4,6,7,Biskra,BISKRA,بسكرة,Biskra
5,8,9,Blida,BLIDA,البليدة,Blida
6,9,10,Bouira,BOUIRA,البويرة,Bouira
7,10,11,Tamenghasset,TAMENGHASSET,تامنغست,Tamanrasset
8,11,12,Tébéssa,TEBESSA,تبسة,Tebessa
9,12,13,Tlemcen,TLEMCEN,تلمسان,Tlemcen
